In [33]:
import numpy as np
from scipy.integrate import quad
from IPython.display import clear_output

def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)

def toBar(Pa):
    return (Pa / 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius
Ppad = 1 # Bar

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, name, omega, Pc, Tb, Tc, NCParams, Ant = None, Hf = 0): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.name = name
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant = Ant
        self.Hf = Hf

    def Antoine(self, T): # NIST, T em K, P em Bar
        AntA = self.Ant[0]
        AntB = self.Ant[1]
        AntC = self.Ant[2]
        if T < self.Ant[3] or T > self.Ant[4]:
            print(f"{self.name} : Temperatura {T} é fora da faixa para antoine")
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura, P em Bar
        Ant = self.Ant
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        return T

    def Estado(self, P, T): # Retorna o estado do composto
        if T > self.Tc and P > self.Pc:
            print(f"{self.name} : Fluído supercrítico")
            return "gas"
        elif T > self.Tc or self.Ant == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if P == Ppad:
                if T > self.Tb:
                    return "gas"
                else:
                    return "liquid"
            if  P < self.Antoine(T):
                return "vapour"
            elif P == self.Antoine(T):
                print(f"{self.name} : FODEO")
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))

    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (1.54226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / toPa(self.Pc))
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / toPa(self.Pc))
        A = (a * P) / ((R ** 2) * (T ** 2))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * ( B ** 2)) - (2 * B)
        gamma = - (A * B) + (B ** 2) + (B ** 3) 
        Z = np.roots([1, alpha, beta, gamma]) # Resolução cúbica
        # print(f"{self.name} Z : {Z}")
        Z = Z[np.isreal(Z)] # elimina raizes imaginarias 
        Z = max(Z) # Usa a raiz mais alta coorespondende ao gás conforme P&R 
        Z = np.real(Z) # ignora a parte imaginaria
        print(f"{self.name} Z : {Z}")
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self): # Por Riedel p147 V.Ness
        return R * self.Tb * ((1.092 * (np.log(self.Pc) - 1.013)) / (0.930 - (self.Tb / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(Ppad, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(P, T): # estado final
                    case "gas" | "vapour":
                        print(f"{self.name} : caminho gás - gás")
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        H = res + ig
                        print(f"{self.name} : Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
                    case "liquid":
                        print(f"{self.name} :caminho gás - líquido")
                        Tvap = self.TAntoine(Ppad)
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        HGas = res + ig
                        Hv = self.Hvap()
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        print(f"{self.name} :T Antoine: {Tvap}, Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, HGas {HGas:.3f} J/mol, Hv {Hv:.3f} J/mol, Hl {Hl:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
            case "liquid":
                match self.Estado(P, T): #estado final
                    case "gas" | "vapour":
                        print(f"{self.name} : caminho líquido - gás")
                        Tvap = self.TAntoine(Ppad)
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        Hv = self.Hvap()
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        HGas = res + ig
                        H = Hl + Hv + HGas
                        print(f"{self.name} : T Antoine: {Tvap}, Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, HGas {HGas:.3f} J/mol, Hv {Hv:.3f} J/mol, Hl {Hl:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
                    case "liquid":
                        print(f"{self.name} : caminho líquido - líquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        print(f"{self.name} : Htotal e Hl {H:.3f} J/mol")
                        return H

# Componentes

C2H4O = Comp( # 
        "Oxido de etileno",
        0.201, # omega Perry
        toBar(7.26e6), # Pc Perry
        toK(10.4), # Tb
        469.15, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [5.84696, 2022.83, 62.656, 273.4, 304.9], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -52630) # Hf J/mol  V. Ness

O2 = Comp( # 
        "Oxigênio",
        0.020, # omega Perry
        toBar(5.02e6), # Pc Perry
        toK(-182.953), # Tb
        154.58, # Tc Perry
        [3.639, # Cp A V.Ness
        0.506e-3, # B
        0, # C
        -0.227e5], # D
        # [3.9523, 340.024, --4.144, 54.36, 154.33], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

N2 = Comp( #
        "Nitrogênio",
        0.037, # omega Perry
        toBar(3.39e6), # Pc Perry
        toK(-195.798), # Tb
        126.2, # Tc Perry
        [3.280, # Cp A V.Ness
        0.593e-3, # B
        0, # C
        0.040e5], # D
        # [3.7362, 264.651, -6.788, 63.14, 126], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

C2H4 = Comp( # 
        "Etileno",
        0.086, # omega Perry
        toBar(5.03e6), # Pc Perry
        toK(-103.8), # Tb
        282.34, # Tc Perry
        [1.424, # Cp A V.Ness
        14.394e-3, # B
        -4.392e-6, # C
        0], # D
        [13.871, 2854.7, -36.23, toK(-169.16), toK(76.52)], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 52510) # Hf J/mol  V. Ness

CO2 = Comp( #
        "Dioxido de carbono",
        0.224, # omega Perry
        toBar(7.39e6), # Pc Perry
        toK(-78.464), # Tb
        304.21, # Tc Perry
        [5.457, # Cp A V.Ness 
        1.045e-3, # B 
        0, # C 
        -1.157e5], # D 
        # [6.81228, 1301.679, -3.494, 154.26, 195.89]   , # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -393509) # Hf J/mol V. Ness 
    
CH3CHO = Comp( # 
        "Acetaldeído",
        0.292, # omega Perry
        toBar(5.57e6), # Pc Perry
        toK(20.6), # Tb Haynes
        466, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [3.68639, 822.894, -69.899, 293.4, 377.5], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -166190) # J/mol V. Ness

H2O = Comp( # 
        "Água",
        0.343, # omega Perry
        toBar(21.94e6), # Pc Perry
        toK(99.974), # Tb Haynes
        647.13, # Tc Perry
        [3.470, # Cp A V.Ness
        1.450e-3, # B
        0, # C
        0.121e5], # D
        [4.6543, 1435.264, -64.848, 255.9, 373], # Antoine A, B, C, D, Tmin, Tmax
        Hf = -285830) # Líquido J/mol Hf V. Ness


class Corrente: # Corrente : Classe que abriga uma corrente toda
    def __init__(self, name, P, T, C2H4O = 0, O2 = 0, N2 = 0, C2H4 = 0, CO2 = 0, H2O = 0, CH3CHO = 0): # pressão, temperatura e vazões molares kmol / h
        self.name =  name
        self.P = P
        self.T = T
        self.C2H4O = C2H4O
        self.O2 = O2
        self.N2 = N2
        self.C2H4 = C2H4
        self.CO2 = CO2
        self.H2O = H2O
        self.CH3CHO = CH3CHO
    def Hcorr(self):
        H = 0
        if self.C2H4O != 0:
            H += self.C2H4O * C2H4O.Hmolar(self.P, self.T)
        if self.O2 != 0:
            H += self.O2 * O2.Hmolar(self.P, self.T)
        if self.N2 != 0:
            H += self.N2 * N2.Hmolar(self.P, self.T)
        if self.C2H4 != 0:
            H += self.C2H4 * C2H4.Hmolar(self.P, self.T)
        if self.CO2 != 0:
            H += self.CO2 * CO2.Hmolar(self.P, self.T)
        if self.H2O != 0:
            H += self.H2O * H2O.Hmolar(self.P, self.T)
        if self.CH3CHO != 0:
            H += self.CH3CHO * CH3CHO.Hmolar(self.P, self.T)
            
        print(f"\n{self.name} : {H:.3f} kJ/h\n")
        return H

In [34]:
# TC-01
C3 = Corrente("Corrente 3, Entrada TC-01", 22, toK(30), 0.000537498,
1399.388446,
5264.911772,
2798.776892,
0.053820258,
0.006353004,
0
)
C4a = Corrente("Corrente 4a, Saída TC-01", 20, toK(140), 0.000537498,
1399.388446,
5264.911772,
2798.776892,
0.053820258,
0.006353004,
0)
C5 = Corrente("Corrente 5, Entrada TC-01", 19, toK(240), 226.3148763,
1102.186328,
5264.911772,
2508.823606,
121.8342004,
121.7867331,
2.748757152
)
C5a = Corrente("Corrente 5a, Saída TC-01", 19, toK(150), 226.3148763,
1102.186328,
5264.911772,
2508.823606,
121.8342004,
121.7867331,
2.748757152)

HC3 = C3.Hcorr()
HC4a = C4a.Hcorr()
HC5 = C5.Hcorr()
HC5a = C5a.Hcorr()
dH34 = HC4a - HC3
dH5 = HC5a - HC5

Difs = {}
for temp in np.arange(30, 240, 0.1):
    C4a.T = toK(temp)
    C5a.T = C4a.T + 10
    dH34 = C4a.Hcorr() - C3.Hcorr()
    dH5 = C5a.Hcorr() - C5.Hcorr()
    dif = dH34 + dH5 
    Difs[temp] = abs(dif)
clear_output(wait=False)
temp = min(Difs, key = Difs.get)
print(temp)
C4a.T = toK(temp) # round(C4a.T, 0)
C5a.T = C4a.T + 10

HC3 = C3.Hcorr()
HC4a = C4a.Hcorr()
HC5 = C5.Hcorr()
HC5a = C5a.Hcorr()
dH34 = HC4a - HC3
dH5 = HC5a - HC5

print(f"C4a.T {C4a.T - 273.15}, Delta C3: {dH34:.3f}, Delta C5 {dH5:.3f} \n dif: {(dH34 + dH5):.3f}")


139.50000000000156
Oxido de etileno :caminho gás - líquido
Oxido de etileno Z : 0.04620692228106952
Oxido de etileno :T Antoine: 283.30668830298134, Hig = 242.319 J/mol, HRes -24219.952 J/mol, HGas -23977.632 J/mol, Hv 25870.130 J/mol, Hl 1798.666 J/mol, Htotal -48049.096 J/mol
Oxigênio : caminho gás - gás
Oxigênio Z : 0.9820556717892159
Oxigênio : Hig = 147.165 J/mol, HRes -210.769 J/mol, Htotal -63.604 J/mol
Nitrogênio : caminho gás - gás
Nitrogênio Z : 0.9926999190022595
Nitrogênio : Hig = 145.609 J/mol, HRes -169.935 J/mol, Htotal -24.327 J/mol
Etileno : caminho gás - gás
Etileno Z : 0.8564370517270636
Etileno : Hig = 222.601 J/mol, HRes -1094.360 J/mol, Htotal -871.759 J/mol
Dioxido de carbono : caminho gás - gás
Dioxido de carbono Z : 0.8785239341787188
Dioxido de carbono : Hig = 186.705 J/mol, HRes -960.985 J/mol, Htotal -774.280 J/mol
Água : caminho líquido - líquido
Água : Htotal e Hl 492.774 J/mol

Corrente 3, Entrada TC-01 : -2657007.238 kJ/h

Oxido de etileno : Temperatura 

In [35]:
# F-01
# QF1 = C4b - C4a
C4b = Corrente("Corrente 4b, Saída Forno", 20, toK(200),  0.000537498,
1399.388446,
5264.911772,
2798.776892,
0.053820258,
0.006353004,
0
)
HC4b = C4b.Hcorr()
QF1 = HC4b - HC4a
print(f"F-01: {QF1} kJ/h")

Oxido de etileno : caminho gás - gás
Oxido de etileno Z : 0.892645498674941
Oxido de etileno : Hig = 10559.488 J/mol, HRes -1312.836 J/mol, Htotal 9246.652 J/mol
Oxigênio : caminho gás - gás
Oxigênio Z : 1.0009919237707332
Oxigênio : Hig = 5344.657 J/mol, HRes -78.257 J/mol, Htotal 5266.400 J/mol
Nitrogênio : caminho gás - gás
Nitrogênio Z : 1.0057017113979354
Nitrogênio : Hig = 5146.510 J/mol, HRes -46.559 J/mol, Htotal 5099.951 J/mol
Etileno : caminho gás - gás
Etileno Z : 0.9728773856459053
Etileno : Hig = 9182.167 J/mol, HRes -450.508 J/mol, Htotal 8731.658 J/mol
Dioxido de carbono : caminho gás - gás
Dioxido de carbono Z : 0.9793842925327486
Dioxido de carbono : Hig = 7333.126 J/mol, HRes -370.947 J/mol, Htotal 6962.180 J/mol
Água : Temperatura 473.15 é fora da faixa para antoine
Água : Temperatura 473.15 é fora da faixa para antoine
Água : caminho líquido - líquido
Água : Htotal e Hl 16463.662 J/mol

Corrente 4b, Saída Forno : 58658979.409 kJ/h

F-01: 22366850.36230237 kJ/h


In [36]:
# R-01
# Reação lei de hess
R01 = (C5.C2H4O - C4b.C2H4O) * C2H4O.Hf + (C5.C2H4 - C4b.C2H4) * C2H4.Hf + (C5.CO2 - C4b.CO2) * CO2.Hf + (C5.H2O - C4b.H2O) * H2O.Hf + (C5.CH3CHO - C4b.CH3CHO) * CH3CHO.Hf
print(f"Reação: {R01} kJ/h")
H45 = HC5 - HC4b
print(f"Delta H, 4b - 5 : {H45} kJ/h")
Hr = R01 + H45
print(f"R-01: {Hr} kJ/h")

Reação: -110323348.30223809 kJ/h
Delta H, 4b - 5 : 20815393.67444799 kJ/h
R-01: -89507954.6277901 kJ/h


In [37]:
# TC-02
# QTC2 = C5b - C5a
C5b = Corrente("Corrente 5a, Saída TC-02", 18, toK(50), 226.3148763,
1102.186328,
5264.911772,
2508.823606,
121.8342004,
121.7867331,
2.748757152
)

HC5b = C5b.Hcorr()
QTC2 = HC5b - HC5a 
print(f"TC-02: {QTC2} kJ/h")

Oxido de etileno : Temperatura 323.15 é fora da faixa para antoine
Oxido de etileno : Temperatura 323.15 é fora da faixa para antoine
Oxido de etileno :caminho gás - líquido
Oxido de etileno Z : 0.5883649107844173
Oxido de etileno :T Antoine: 283.30668830298134, Hig = 1248.458 J/mol, HRes -3003.001 J/mol, HGas -1754.543 J/mol, Hv 25870.130 J/mol, Hl 3676.731 J/mol, Htotal -23947.941 J/mol
Oxigênio : caminho gás - gás
Oxigênio Z : 0.9888806987724332
Oxigênio : Hig = 740.108 J/mol, HRes -154.599 J/mol, Htotal 585.509 J/mol
Nitrogênio : caminho gás - gás
Nitrogênio Z : 0.996682263191082
Nitrogênio : Hig = 728.707 J/mol, HRes -122.475 J/mol, Htotal 606.232 J/mol
Etileno : caminho gás - gás
Etileno Z : 0.9057082791596389
Etileno : Hig = 1137.299 J/mol, HRes -783.636 J/mol, Htotal 353.663 J/mol
Dioxido de carbono : caminho gás - gás
Dioxido de carbono Z : 0.9209671661275187
Dioxido de carbono : Hig = 952.165 J/mol, HRes -685.749 J/mol, Htotal 266.416 J/mol
Água : caminho líquido - líquido
Ág

In [40]:
# Dest-01
# Roult na corrente de fundo
T12 = C2H4O.TAntoine(5.8)
C12 = Corrente("Corrente 12, Entrada", 5.8, T12, C2H4O = 225.1833014, H2O = 65.83685424, CH3CHO = 2.748757152)
print(f"T12 {T12}")
T15 = C2H4O.TAntoine(5.4) + 0.000001
C15 = Corrente("Corrente 15, Topo", 5.4, T15, C2H4O = 675.5499055, CH3CHO = 5.447983115) 
print(f"T15 {T15}")
T16 = 430.215
C16 = Corrente("Corrente 16, Fundo", 5.6, T16, H2O = 65.83685424, CH3CHO = 0.932762781) # NÃO USAR RAOULT?
print(f"T16 {T16}")
P15d =  C2H4O.Antoine(toK(60)) + 0.000001
C15d = Corrente("Corrente 15d, Saída OE", P15d, toK(60), C2H4O = 225.1833018, CH3CHO = 1.815994372)
C15a = Corrente("Corrente 15a, Após TC-06", P15d, T15, C2H4O = 675.5499055, CH3CHO = 5.447983115) 
print(f"P 15d {P15d}")

HC12 = C12.Hcorr()
HC15d = C15d.Hcorr()
HC16 = C16.Hcorr()

HC15 = C15.Hcorr() 
HC15a = C15a.Hcorr()

# C15c - C15 = TC-06
TC06 = HC15a - HC15 

# C15d + C16 - C12 = Q, Q = TC-06 + TC-07
Hdest = HC15d + HC16 - HC12
TC07 = Hdest - TC06

print(f"Hdest: {Hdest} kJ/h, TC-06: {TC06} kJ/h, TC-07: {TC07} kJ/h")


T12 335.2622185611451
T15 332.8477260490245
T16 430.215
Oxido de etileno : Temperatura 333.15 é fora da faixa para antoine
P 15d 5.448786694239585
Oxido de etileno : Temperatura 335.2622185611451 é fora da faixa para antoine
Oxido de etileno : Temperatura 335.2622185611451 é fora da faixa para antoine
Oxido de etileno :caminho gás - líquido
Oxido de etileno Z : 0.9161559969843605
Oxido de etileno :T Antoine: 283.30668830298134, Hig = 1886.090 J/mol, HRes -623.157 J/mol, HGas 1262.933 J/mol, Hv 25870.130 J/mol, Hl 4850.008 J/mol, Htotal -19757.189 J/mol
Água : caminho líquido - líquido
Água : Htotal e Hl 3603.021 J/mol
Acetaldeído :caminho gás - líquido
Acetaldeído Z : 0.8853750927957077
Acetaldeído :T Antoine: 293.1238893904335, Hig = 1886.090 J/mol, HRes -887.030 J/mol, HGas 999.060 J/mol, Hv 26765.320 J/mol, Hl 4355.261 J/mol, Htotal -21410.998 J/mol

Corrente 12, Entrada : -4270631.052 kJ/h

Oxido de etileno : Temperatura 333.15 é fora da faixa para antoine
Oxido de etileno : Temper

In [39]:
# Utilidades
# Água de resfriamento
agua = H2O.Hmolar(Ppad, toK(40))

Água : caminho líquido - líquido
Água : Htotal e Hl 1470.912 J/mol
